In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=4, app_name='pdu_syp2', node_spec=NODE_MEDIUM)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 24G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:27


성장형 무기 소지자가, 해당 스킨을 안쓰고 다른 스킨을 쓰는 경우가 얼마나 되는지입니다. (그리고 트렌드도...)
성장형 스킨을 뽑은 사람은 잠시 그것을 쓰겠지만, 철이 지나고 나면 다른 일반 고급스킨으로 갈아타는 패턴이 생겼으면 좋겠다고 생각하는데, 과연 그렇게 흐르는지 알고 싶어요.
성장형 무기가 장기적으로 다른 스킨 매출에 영향을 주는지를 유추해볼 수 있지 않을까 싶습니다.

In [63]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [4]:
gcoin = load_data_mart(start_date="2021-08-11", end_date="2021-09-07", device="pc", table_name="gcoin_use")

In [5]:
# 8월 성장형 무기 스킨 출시 후 2주간 contraband로 아이템 획득

device = 'pc' 
env = 'live' 
log_name = 'ItemUpgraded'
start_date = '2021-08-11'
end_date = '2021-08-24' 
item_upgrade = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [6]:
# 8월 성장형 무기 스킨 출시 후 4주간 아이템 착용

device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-08-11'
end_date = '2021-09-07' 
item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [7]:
items = mysql.read_table(spark, "metainfo", "item_meta_sales")
wsus_product_id = items.where(lower(col("product_name")).like("%progressive%")).select(col("product_id")).distinct().toPandas()["product_id"].tolist()

In [8]:
wsus_product_id

['itemdesc.12012001',
 'itemdesc.12012004',
 'itemdesc.12012002',
 'itemdesc.12012003']

In [9]:
# 성장형 무기를 획득한 사람들, 
# ItemDescId, AccountId
wsus_account = item_upgrade.where(col("ItemDescId").isin(wsus_product_id)).select("ItemDescId", "AccountId").distinct().toPandas()

In [10]:
wsus_account.shape

(74990, 2)

In [11]:
wsus_account.head()

,ItemDescId,AccountId
0,itemdesc.12012002,account.1d55a34a2a774681b883dd03524adbb8
1,itemdesc.12012002,account.48b51939b27f4bc18cdbdf9e25cbe642
2,itemdesc.12012002,account.40292352c9c04790866ea12de51ede06
3,itemdesc.12012002,account.3c2bbb65bd6f4ed894397432d452c3e0
4,itemdesc.12012002,account.ec274e03754e4d4c8ed9cd6c8a7fc1b5


In [12]:
aug_wsus_account_list = wsus_account[wsus_account.ItemDescId == "itemdesc.12012002"].AccountId.tolist()

In [13]:
len(aug_wsus_account_list)

60626

In [14]:
wsus_account[wsus_account.AccountId.isin(aug_wsus_account_list)].groupby("AccountId", as_index=False)["ItemDescId"].count().describe()

,ItemDescId
count,60626.000000
mean,1.137169
std,0.344028
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [15]:
# 성장형 무기 보유자들의 성장형 무기 보유 개수
wsus_cnt = item_upgrade.where((col("ItemDescId").isin(wsus_product_id))).groupBy("AccountId").agg(countDistinct("ItemDescId")).toPandas()

In [16]:
wsus_cnt.describe()

,count(DISTINCT ItemDescId)
count,66674.000000
mean,1.124726
std,0.330411
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [17]:
weapon_product_list = items.where(col("sub_category") == "weapon").select(col("product_id")).distinct().toPandas().product_id.tolist()

In [18]:
tmp = item_equipped.where((col("AccountId").isin(aug_wsus_account_list)) & (col("ItemId").isin(weapon_product_list))).toPandas()

In [19]:
tmp.head()

,date,AccountId,ItemId,PartId,PlatformProvider,Provider,Time,Type
0,2021-08-11,account.ad2d7799560c49b180642a35e23b70ec,itemdesc.12010387,partdesc.machete,PcSteam,steam,2021-08-11T13:19:45.4932794Z,ItemEquipped
1,2021-08-11,account.2bcf86bbf37b4c4fb75a89ab3b1b365f,itemdesc.12012001,partdesc.m762,PcSteam,steam,2021-08-11T13:19:54.6336698Z,ItemEquipped
2,2021-08-11,account.ca99dd3e2c29477793c7e5ecc73db5ab,itemdesc.12012002,partdesc.fnfal,PcSteam,steam,2021-08-11T13:22:36.9401208Z,ItemEquipped
3,2021-08-11,account.5faab55af04141b8be2cdae61ad55fc4,itemdesc.12012002,partdesc.fnfal,PcSteam,steam,2021-08-11T13:22:56.6166484Z,ItemEquipped
4,2021-08-11,account.3011f8ad49354f3d93ffb046a45250e1,itemdesc.12012002,partdesc.fnfal,PcSteam,steam,2021-08-11T13:10:59.7230886Z,ItemEquipped


In [20]:
aug_wsus_equip_account_list = tmp[(tmp.AccountId.isin(aug_wsus_account_list)) & (tmp.ItemId == "itemdesc.12012002")].AccountId.unique().tolist()

In [21]:
len(aug_wsus_equip_account_list)

60106

60106명 대하여 기간동안 사용한 무기 스킨 수

In [22]:
tmp[tmp.AccountId.isin(aug_wsus_equip_account_list)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,60106.000000
mean,2.397714
std,2.548117
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,49.000000


In [23]:
# 8월 성장형 무기 소유자 & 이용자(60106)에 대한 무기에 대해 소비
gcoin_use = gcoin.where((col("account_id").isin(aug_wsus_equip_account_list)) & (col("product_id").isin(weapon_product_list))).groupBy("account_id", "product_id").agg((sum("paid_use")/100).alias("paid")).toPandas()

그래프 용
- 일별 사용 무기 스킨
- 무기 스킨별 매출
- slr 무기 스킨별 매출

In [26]:
aug_wsus_item_equipped = item_equipped.where((col("AccountId").isin(aug_wsus_account_list)) & (col("ItemId").isin(weapon_product_list))).groupBy("date", "AccountId", "ItemId").agg(count("*").alias("cnt"))
product_name = items.select("product_id", "product_name").distinct()
aug_wsus_item_equipped_table = aug_wsus_item_equipped.join(product_name, aug_wsus_item_equipped.ItemId == product_name.product_id, "left")
mysql.insert_table(aug_wsus_item_equipped_table, "labs", "aug_wsus_item_equipped_table")

In [27]:
aug_wsus_item_equipped_table.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- cnt: long (nullable = false)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)



In [28]:
aug_wsus_item_equipped_table.groupBy("product_name").agg(sum("cnt").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,[PROGRESSIVE] Buzzkill - SLR,97959
1,[PROGRESSIVE] Gear Head - Beryl M762,21098
2,Gold Plate - Beryl M762,6995
3,Silver Plate - M416,4950
4,LongSkr's AKM,4600
5,Son Heung-min - M416,4247
6,CNH's SLR,4234
7,Silver Plate - Beryl M762,3541
8,Gold Plate - M416,3342
9,Seahorse Cult - Beryl M762,2692


In [29]:
aug_wsus_item_equipped_table.groupBy("product_name").agg(countDistinct("AccountId").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,[PROGRESSIVE] Buzzkill - SLR,60106
1,[PROGRESSIVE] Gear Head - Beryl M762,10715
2,Gold Plate - Beryl M762,3800
3,LongSkr's AKM,2968
4,CNH's SLR,2605
5,Son Heung-min - M416,2589
6,Silver Plate - M416,2256
7,Gold Plate - M416,1989
8,Seahorse Cult - Beryl M762,1849
9,Silver Plate - Beryl M762,1672


In [30]:
aug_wsus_gcoin_table = gcoin.where((col("account_id").isin(aug_wsus_equip_account_list)) & (col("product_id").isin(weapon_product_list))).select("date", "account_id", "product_id", "free_use", "paid_use", "currency", "product_name")
mysql.insert_table(aug_wsus_gcoin_table, "labs", "aug_wsus_gcoin_table")

In [31]:
aug_wsus_gcoin_table.printSchema()

root
 |-- date: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- free_use: long (nullable = true)
 |-- paid_use: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- product_name: string (nullable = true)



In [32]:
aug_wsus_gcoin_table.groupBy("product_name").agg((sum("paid_use")/100).alias("revenue")).orderBy("revenue", ascending=False).toPandas().head(20)

,product_name,revenue
0,Son Heung-min - M416,5034.4
1,LongSkr's AKM,5014.9
2,CNH's SLR,3445.7
3,Seahorse Cult - Beryl M762,2975.0
4,Colorful Chaos - Mk14,2393.9
5,Son Heung-min - Mini14,1629.5
6,Manticore - AKM,1120.6
7,Bunnycomb - M24,1089.9
8,Manticore - SLR,1076.0
9,Blue Whale Cult - AKM,943.2


In [42]:
aug_slr_gcoin_table = gcoin.where((col("account_id").isin(aug_wsus_account_list)) & (col("product_id").isin(slr_skin_list))).select("date", "account_id", "product_id", "free_use", "paid_use", "currency", "product_name")
mysql.insert_table(aug_slr_gcoin_table, "labs", "aug_slr_gcoin_table")

In [43]:
aug_slr_gcoin_table.groupBy("product_name").agg((sum("paid_use")/100).alias("revenue")).orderBy("revenue", ascending=False).toPandas().head(20)

,product_name,revenue
0,CNH's SLR,3550.8
1,Manticore - SLR,1090.3
2,Harlequin - SLR,661.3


In [44]:
aug_slr_gcoin_table.groupBy("product_name").agg(countDistinct("account_id").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,CNH's SLR,417
1,Manticore - SLR,183
2,Harlequin - SLR,76


In [34]:
tmp2 = pd.merge(tmp, gcoin_use, left_on=["AccountId", "ItemId"], right_on=["account_id", "product_id"], how="left").drop(["PlatformProvider", "Provider", "Type", "account_id", "product_id"], axis=1)

8월 성장형 무기 사용한 사람들 대상 (aug_wsus_equip_account_list) slr무기에 대한 스킨 착용

In [35]:
slr = tmp2[(tmp2.AccountId.isin(aug_wsus_equip_account_list)) & (tmp2.PartId == "partdesc.fnfal")].sort_values(["AccountId", "date"]).groupby(["date", "AccountId", "ItemId"], as_index=False)["paid"].sum().sort_values(["AccountId", "date"])

In [36]:
slr_skin_list = slr.ItemId.unique().tolist()

In [40]:
items.where(col("product_id").isin(slr_skin_list)).select("product_name", "product_id").distinct().show(truncate=False)

+---------------------------------+-----------------+
|product_name                     |product_id       |
+---------------------------------+-----------------+
|ReleeN's SLR                     |itemdesc.12010179|
|PCS3 Blue Bullion - SLR          |itemdesc.12010340|
|fuffenz's SLR                    |itemdesc.12010075|
|Manticore - SLR                  |itemdesc.12010348|
|CNH's SLR                        |itemdesc.12010439|
|[PROGRESSIVE] Buzzkill - SLR     |itemdesc.12012002|
|Mazarin1k's SLR                  |itemdesc.12010138|
|PGC 2019 Gen.G Championship - SLR|itemdesc.12010350|
|Mithrain's SLR                   |itemdesc.12010178|
|Harlequin - SLR                  |itemdesc.12010417|
|BreaK's SLR                      |itemdesc.12010220|
|Swagger's SLR                    |itemdesc.12010171|
|SkipNhO's SLR                    |itemdesc.12010064|
+---------------------------------+-----------------+



In [45]:
slr.groupby(["AccountId"])["ItemId"].nunique().reset_index().describe()

,ItemId
count,60106.000000
mean,1.107976
std,0.378774
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,10.000000


In [46]:
slr.groupby(["AccountId"])["ItemId"].nunique().reset_index().groupby("ItemId", as_index=False)["AccountId"].count()

,ItemId,AccountId
0,1,54699
1,2,4558
2,3,678
3,4,130
4,5,28
5,6,7
6,7,5
7,10,1


slr 스킨 1개 사용한 사람은 12012002만 사용한게 맞는지 확인

In [47]:
a = slr.groupby(["AccountId"])["ItemId"].nunique().reset_index()

In [48]:
slr[slr.AccountId.isin(a[a.ItemId == 1].AccountId.unique().tolist())].ItemId.unique()

array(['itemdesc.12012002'], dtype=object)

In [49]:
sskin_cnt = slr.groupby(["AccountId"])["ItemId"].nunique().reset_index()

In [50]:
sskin2 = sskin_cnt[sskin_cnt.ItemId == 2].AccountId.unique().tolist()
sskin3 = sskin_cnt[sskin_cnt.ItemId == 3].AccountId.unique().tolist()
sskin4 = sskin_cnt[sskin_cnt.ItemId == 4].AccountId.unique().tolist()
sskin5 = sskin_cnt[sskin_cnt.ItemId >= 5].AccountId.unique().tolist()

In [51]:
slr[(slr.AccountId.isin(sskin2)) & (slr.paid>0)].AccountId.nunique()

501

In [52]:
slr[(slr.AccountId.isin(sskin2)) & (slr.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,501.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [54]:
slr[(slr.AccountId.isin(sskin2)) & (slr.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010348    132
itemdesc.12010417     49
itemdesc.12010439    320
Name: AccountId, dtype: int64

In [55]:
slr[(slr.AccountId.isin(sskin3)) & (slr.paid>0)].AccountId.nunique()

34

In [56]:
slr[(slr.AccountId.isin(sskin3)) & (slr.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,34.000000
mean,1.029412
std,0.171499
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [57]:
slr[(slr.AccountId.isin(sskin3)) & (slr.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010348     7
itemdesc.12010417     6
itemdesc.12010439    22
Name: AccountId, dtype: int64

In [58]:
slr[(slr.AccountId.isin(sskin4)) & (slr.paid>0)].AccountId.nunique()

13

In [59]:
slr[(slr.AccountId.isin(sskin4)) & (slr.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,13.000000
mean,1.153846
std,0.375534
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [60]:
slr[(slr.AccountId.isin(sskin4)) & (slr.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010348    3
itemdesc.12010417    5
itemdesc.12010439    7
Name: AccountId, dtype: int64

In [61]:
slr[(slr.AccountId.isin(sskin5)) & (slr.paid>0)].AccountId.nunique()

0

In [62]:
slr[(slr.AccountId.isin(sskin5)) & (slr.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [8]:
548/60106*100

0.9117226233653878